In [1]:
# from langchain.callbacks.base import BaseCallbackHandler

# class CustomCallbackHandler(BaseCallbackHandler):
#     def on_tool_start(self, tool_name: str, **kwargs):
#         print(f"Tool {tool_name} started with arguments: {kwargs}")

#     def on_tool_end(self, tool_name: str, output: str, **kwargs):
#         print(f"Tool {tool_name} finished with output: {output}")

#     def on_llm_start(self, **kwargs):
#         print("LLM started")

#     def on_llm_end(self, output: str, **kwargs):
#         print(f"LLM finished with output: {output}")

# # Example usage
# handler = CustomCallbackHandler()

# # Simulating some events
# handler.on_tool_start("example_tool", arg1="value1", arg2="value2")
# handler.on_tool_end("example_tool", output="tool output")
# handler.on_llm_start()
# handler.on_llm_end(output="LLM output")


# Dependencies

In [2]:
# !pip install Openai

In [3]:
# !apt install libgraphviz-dev
# !pip install pygraphviz

In [4]:
# !pip install zigzag

In [5]:
# !pip install influxdb_client

In [6]:
# !pip install import_ipynb

In [7]:
# !pip install crewai

In [8]:
# !pip install unstructured

In [9]:
# !pip install langgraph langchain langchain_openai langchainhub langchain-core langchain_experimental

In [10]:
# !pip install sec_api

In [11]:
# !pip install langchain_groq

In [12]:
# !pip install groq

In [13]:
# !pip install git+https://github.com/saeid976/researcher

# Function_call notebook

In [14]:
# %run functions_python.ipynb

In [15]:
# fc = FunctionCalls()

In [16]:
import functions_python
import input_filter

In [17]:
# fc = functions_python.FunctionCalls()

# Langgraph

In [18]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
from langgraph.prebuilt import ToolExecutor
from typing import Optional, Type

## On boarding:

In [19]:
# !pip install psycopg2

In [20]:
# import psycopg2

In [21]:
# class SaveTradingStyle(BaseModel):
#     """save the style of trading."""

#     userId: float = Field(..., description="user id")


# def trading_style_saver(userId: float) -> str:
#     try:
#         # Connect to the PostgreSQL server
#         conn = psycopg2.connect(
#             dbname="tensurf",
#             user="tensurf",
#             password="tensurf",
#             host="135.181.158.245",
#             port="5432"
#         )

#         # Create a cursor object
#         cur = conn.cursor()

#         trading_style = input("What is your trading style (day trader, swing trader, or both)? ")
#         cur.execute("SELECT COUNT(*) FROM onboarding_user WHERE id = %s",(userId,))
#         count = cur.fetchone()[0]
#         if count > 0:
#             cur.execute("UPDATE onboarding_user SET trading_style=%s WHERE id=%s",(trading_style, userId))
#         else:
#             cur.execute("INSERT INTO onboarding_user (id,trading_style) values (%s,%s)", (userId, trading_style))

#         conn.commit()
#         cur.close()
#         conn.close()

#         return "Trading style is saved successfully."

#     except psycopg2.Error as e:
#         return f"Error in trading_style_saver: {e}"


# databaseStyle = StructuredTool.from_function(
#     func=trading_style_saver,
#     name="SaveTradingStyle",
#     description="save the style of trading",
#     args_schema=SaveTradingStyle,
#     return_direct=False,
# )

In [22]:
# class SaveRiskPercentage(BaseModel):
#     """Get the info of the trader about how much risk they take for trading."""

#     userId: float = Field(..., description="user id")

# def risk_percentage_saver(userId) -> str:
#     # Save risk percentage to the database
#     try:
#         # Connect to the PostgreSQL server
#         conn = psycopg2.connect(
#             dbname="tensurf",
#             user="tensurf",
#             password="tensurf",
#             host="135.181.158.245",
#             port="5432"
#         )

#         # Create a cursor object
#         cur = conn.cursor()

#         cur.execute("SELECT COUNT(*) FROM onboarding_user WHERE id = %s",(userId,))
#         while True:
#             percentage = float(input("What is the maximum percentage of your capital that you want to risk in trade (0-100)? "))
#             if 0 <= percentage <= 100:
#                 break
#             else:
#                 print("Please enter a positive float number between 0 and 100.")
#         count = cur.fetchone()[0]
#         if count > 0:
#             cur.execute("UPDATE onboarding_user SET risk_parameter=%s WHERE id=%s",(percentage, userId))
#         else:
#             cur.execute("INSERT INTO onboarding_user (id,risk_parameter) values (%s,%s)", (userId, percentage))

#         # Commit the transaction
#         conn.commit()
#         cur.close()
#         conn.close()

#         return "percentage of risk is saved successfully."

#     except psycopg2.Error as e:
#         return f"Error in risk_percentage_saver: {e}"


# databaseRisk = StructuredTool.from_function(
#     func=risk_percentage_saver,
#     name="SaveRiskPercentage",
#     description="Get the info of the trader about how much risk they take for trading.",
#     args_schema=SaveRiskPercentage,
#     return_direct=False,
# )

In [23]:
# def check_user_id():
#     try:
#         # Connect to the PostgreSQL server
#         conn = psycopg2.connect(
#             dbname="tensurf",
#             user="tensurf",
#             password="tensurf",
#             host="135.181.158.245",
#             port="5432"
#         )

#         # Create a cursor object
#         cur = conn.cursor()
#         user_id = input("What is your id?")
#         # Query to check if the user_id exists in the table
#         cur.execute("SELECT COUNT(*) FROM onboarding_user WHERE id = %s",(user_id,))
#         count = cur.fetchone()[0]

#         if count > 0:
#           return user_id

#         else:
#           return f"User with user_id= {user_id} does not exist in the table."

#     except psycopg2.Error as e:
#       return f"Error in check_user_id: {e}"

# databaseId = StructuredTool.from_function(
#     func=check_user_id,
#     name="CheckUserId",
#     description="Checking the id of the user",
#     return_direct=False,
# )

## Calculate Stop Loss:


In [24]:
class PropertiesCalculateSl(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")

	method: Optional[str] = Field(None, description="shows the method of SL calculation.")

	direction: Optional[int] = Field(None, description="-1: means the user want to calculate stoploss for a short position. 1: means the user want to calculate stoploss for a long position")

	lookback: Optional[int] = Field(None, description="it is used when the method is set to 'minmax' and shows the number of candles that the SL is calculated based on them.")

	neighborhood: Optional[int] = Field(None, description="A parameter that is used in the swing method to define the range or window within which swings are detected. example: \
If the 'neighborhood' parameter is set to 3, it means that the swing detection is based on considering 3 candles to the \
left and 3 candles to the right of the swing point.")

	atr_coef: Optional[int] = Field(None, description="it is used if the method is 'atr' and shows the coefficient of atr")


class CalculateSL(BaseTool):
	name = "calculate_sl"
	description = """Stoploss (SL) is a limitation for potential losses in a position. It's below the current price for long position and above it for short position. \
Distance between the SL and current price is named risk value. This function calculates the SL based o some different methods. \
Returns A dictionary same as this: \
{'sl': [17542.5], 'risk': [268.5], 'info': ['calculated based on maximum high price of previous 100 candles']} \
which includes sl value, risk on the trade and an information. \
If user don't select any method for sl calculation or select "level" method, or zigzag method the otput can include \
more than one stoploss and the values type in the output can be a list such as this \
{'sl': [17542.5, 17818.25, 17858.5, 17882.5, 18518.75], 'risk': [268.5, 7.25, 47.5, 71.5, 707.75], 'info': ['minmax', 'swing', 'atr', '5min_SR', 'daily_SR']} \
It includes a list of stoplosses and the risk on them and finally the level or method name of stoploss."""

	args_schema: Type[BaseModel] = PropertiesCalculateSl

	def _run(
        self, symbol: str = None, method: str = None, direction: int = None,
		lookback: int = None, neighborhood: int = None, atr_coef: int = None
    ) -> int:
		parameters = {
                      "symbol": symbol,
                      "method": method,
                      "direction": direction,
                      "lookback": lookback,
                      "neighborhood": neighborhood,
                      "atr_coef": atr_coef
                      }
		
		fc = functions_python.FunctionCalls()
		
		return fc.calculate_sl(parameters)

SL = CalculateSL()

## Calculate Take-Profit:

In [25]:
class PropertiesCalculateTp(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")

	direction: Optional[int] = Field(None, description="-1: means the user want to calculate stoploss for a short position. 1: means the user want to calculate stoploss for a long position")

	stoploss: Optional[int] = Field(None, description="the value for stoploss")


class CalculateTp(BaseTool):
	name = "calculate_tp"
	description = """Take profit (TP) is opposite of the stop-loss (SL) and is based on maximum reward that we intend to achieve from a trade. \
It represents the price level at which a trader aims to close a position to secure profits before the market reverses. \
Returns list of price for take-profit and information for each price For exampe: \
{'tp': [5139.25, 5140.25, 5144.0], 'info': ['calculated based on the level VWAP_Top_Band_2', 'calculated based on the level Overnight_high', 'calculated based on the level VWAP_Top_Band_3']}"""

	args_schema: Type[BaseModel] = PropertiesCalculateTp

	def _run(
        self, symbol: str = None, direction: int = None, stoploss: int = None
    ) -> int:
		parameters = {
						"symbol": symbol,
						"direction": direction,
						"stoploss": stoploss
						}

		fc = functions_python.FunctionCalls()
		
		return fc.calculate_tp(parameters)

TP = CalculateTp()

## Trend Detection:

In [26]:
class PropertiesCalculateTrend(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")

	start_datetime: Optional[str] = Field(None, description="The start timestamp of period over which the analysis is done. \
	The format of the date should be in the following format %b-%d-%y %H:%M:%S like this example: May-1-2024 13:27:49")
	end_datetime: Optional[str] = Field(None, description="The end timestamp of period over which the analysis is done. \
	The format of the date should be in the following format %b-%d-%y %H:%M:%S like this example: May-1-2024 13:27:49. \
	The user can set this parameter to now. In this situation this parameter's value is the current date time.")

	lookback: Optional[str] = Field(None, description="The number of seconds, minutes, hours, days, weeks, months or years to look back for calculating the trend of the given symbol. \
This parameter determines the depth of historical data to be considered in the analysis. The format of this value must obey one of the following examples: 30 seconds, 10 minutes, 2 hours, 5 days, 3 weeks, 2 months and 3 years. \
Either start_datetime along with end_datetime should be specified or lookback should be specified but both cases should not happen simultaneously.")

class CalculateTrend(BaseTool):
	name = "detect_trend"
	description = """Analyzes the trend of a specified financial instrument over a given time range. \
It is designed primarily for financial data analysis, enabling users to gauge the general direction of a security or market index. \
Whether start_datetime with end_datetime, end_datetime with lookback or lookback parameters could be valued for determining the period over which's trend wants to be detected. \
The function returns a numerical value that indicates the trend intensity and direction within the specified parameters. \
Returns a number between -3 and 3 that represents the trend’s intensity and direction. The value is interpreted as follows: \
\n -3: strong bearish (downward) trend \
\n -2: moderate bearish (downward) trend \
\n -1: mild bearish (downward) trend \
\n 0: without significant trend \
\n 3: strong bullish (upward) trend \
\n 2: moderate bullish (upward) trend \
\n 1: mild bullish (upward) trend"""

	args_schema: Type[BaseModel] = PropertiesCalculateTrend

	def _run(
			self, symbol: str = None, start_datetime: str = None, end_datetime: str = None, lookback: str = None
    ) -> dict:

		function_arguments = {
						"symbol": symbol,
						"start_datetime": start_datetime,
						"end_datetime": end_datetime,
						"lookback": lookback
						}

		fc = functions_python.FunctionCalls()
		
		return fc.detect_trend(function_arguments)


Trend = CalculateTrend()

## Calculate Support and Resistance Levels:

In [27]:
class PropertiesCalculateSR(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")
	timeframe: Optional[str] = Field(None, description="Specifies the timeframe of the candlestick chart to be analyzed. \
	This parameter defines the granularity of the data used for calculating the levels. The only allowed formats would like 3h, 20min, 1d.")
	lookback_days: Optional[str] = Field(None, description="The number of days to look back for calculating the support and resistance levels. \
This parameter determines the depth of historical data to be considered in the analysis. (e.g. 10 days)")

class CalculateSR(BaseTool):
	name = "calculate_sr"
	description = """Support and resistance levels represent price points on a chart where the odds favor a pause or reversal of a prevailing trend. \
This function analyzes candlestick charts over a specified timeframe and lookback period to calculate these levels and their respective strengths. \
Returns a dictionary containing five lists, each corresponding to a specific aspect of the calculated support and resistance levels: \
1. levels_prices (list of floats): The prices at which support and resistance levels have been identified. \
2. levels_start_timestamps (list of timestamps) \
3. levels_detect_timestamps (list of timestamps) \
4. levels_end_timestamps (list of timestamps) \
5. levels_scores (list of floats): Scores associated with each level, indicating the strength or significance of the level. Higher scores typically imply stronger levels."""

	args_schema: Type[BaseModel] = PropertiesCalculateSR

	def _run(
			self, symbol: str = None, timeframe: str = None, lookback_days: str = None
    ) -> dict:
		parameters = {
						"symbol": symbol,
						"timeframe": timeframe,
						"lookback_days": lookback_days
						}

		fc = functions_python.FunctionCalls()
		
		return fc.calculate_sr(parameters)

SR = CalculateSR()

## Bias detection

In [28]:
class PropertiesBiasDetection(BaseModel):
	symbol: Optional[str] = Field(None, description="The ticker symbol of the financial instrument to be analyzed.")

	method: Optional[str] = Field(None, description="The user can choose from different methods including MC, Zigzag trend, \
Trend detection, weekly wvap, candle stick pattern, cross ma, vp detection ,power & counter ratio.")

class BiasDetection(BaseTool):
	name = "bias_detection"
	description = """Detecting trading bias through different methods or Detecting the appropriate entry point for a long or short trade.
Returns a number between -3 and 3 that represents the trend’s intensity and direction. The value is interpreted as follows:
-3: Strong downward , -2: downward -1: Weak downward, 0: No significant trend / Neutral, 1: Weak upward, 2.upward, 3: Strong upward"""

	args_schema: Type[BaseModel] = PropertiesBiasDetection

	def _run(
        self, symbol: str = None, method: str = None
    ) -> dict:

		parameters = {
						"symbol": symbol,
						"method": method
						}

		fc = functions_python.FunctionCalls()
		
		return fc.get_bias(parameters)

Bias = BiasDetection()

## Financial tools

In [29]:
# import os
# os.environ['AZURE_EMBEDDING_MODEL'] ='embedding-ada-002'
# os.environ['LLM_PROVIDER'] = 'groq'
# os.environ["AZURE_OPENAI_API_KEY"] =  "80ddd1ad72504f2fa226755d49491a61"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://tensurfbrain1.openai.azure.com/"
# os.environ["OPENAI_API_VERSION"] = "2023-10-01-preview"
# os.environ['EMBEDDING_PROVIDER']='azureopenai'
# os.environ['SMART_LLM_MODEL']='llama3-70b-8192'
# os.environ['TAVILY_API_KEY']='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'
# os.environ['GROQ_API_KEY'] = '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'

In [30]:
# from gpt_researcher import GPTResearcher
# import asyncio
# from dotenv import load_dotenv
# import os

In [31]:
# query = "what day is today?"
# report_type = "research_report"

# researcher = GPTResearcher(query, report_type)
# report = await researcher.conduct_research()
# report

In [32]:
# from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool
# from langchain.text_splitter            import CharacterTextSplitter
# from langchain.embeddings               import AzureOpenAIEmbeddings
# from langchain_community.vectorstores   import FAISS
# from unstructured.partition.html        import partition_html
# from sec_api                            import QueryApi
# import requests
# import json

In [33]:
# api_type = "azure"
# api_endpoint = 'https://tensurfbrain1.openai.azure.com/'
# api_version = '2023-10-01-preview'
# api_key = '80ddd1ad72504f2fa226755d49491a61'

In [34]:
# import os
# from openai import AzureOpenAI

In [35]:
# api_type = "azure"
# api_endpoint = 'https://tensurfbrain1.openai.azure.com/'
# api_version = '2023-10-01-preview'
# api_key = '80ddd1ad72504f2fa226755d49491a61'

In [36]:
# client = AzureOpenAI(
#     api_key=api_key,
#     api_version=api_version,
#     azure_endpoint=api_endpoint
# )

In [37]:
# def generate_embeddings(text, model='embedding-ada-002'):
#   return client.embeddings.create(input = [text], model=model).data[0].embedding

### Search tools:

In [38]:
# class PropertiesSearchTheInternet(BaseModel):
#     query: str = Field(..., description="A random topic that will be searched on the internet")


# class SearchTheInternet(BaseTool):
#     name = "SearchTheInternet"
#     description = """Useful to search the internet about a a given topic and return relevant results"""

#     args_schema: Type[BaseModel] = PropertiesSearchTheInternet

#     def _run(
#         self, query: str
#     ) -> dict:
#             top_result_to_return = 4
#             url = "https://google.serper.dev/search"
#             payload = json.dumps({"q": query})
#             headers = {
#                 'X-API-KEY': '77976766163a61a4bae1ed8672ae79e916955783',
#                 'content-type': 'application/json'
#             }
#             response = requests.request("POST", url, headers=headers, data=payload)
#             results = response.json()['organic']
#             string = []
#             for result in results[:top_result_to_return]:
#               try:
#                 string.append('\n'.join([
#                     f"Title: {result['title']}", f"Link: {result['link']}",
#                     f"Snippet: {result['snippet']}", "\n-----------------"
#                 ]))
#               except KeyError:
#                 next

#             return '\n'.join(string)

# SearchInternet = SearchTheInternet()

In [39]:
# SearchInternet.run("Apple")

In [40]:
# class PropertiesSearchNews(BaseModel):
#     query: str = Field(..., description="A random topic that will be searched on the internet")


# class SearchNews(BaseTool):
#     name = "SearchNews"
#     description = """Useful to search news about a company, stock or any other topic and return relevant results"""

#     args_schema: Type[BaseModel] = PropertiesSearchNews

#     def _run(
#         self, query: str
#     ) -> dict:
#             top_result_to_return = 4
#             url = "https://google.serper.dev/news"
#             payload = json.dumps({"q": query})
#             headers = {
#                 'X-API-KEY': '77976766163a61a4bae1ed8672ae79e916955783',
#                 'content-type': 'application/json'
#             }
#             response = requests.request("POST", url, headers=headers, data=payload)
#             results = response.json()['news']
#             string = []
#             for result in results[:top_result_to_return]:
#               try:
#                 string.append('\n'.join([
#                     f"Title: {result['title']}", f"Link: {result['link']}",
#                     f"Snippet: {result['snippet']}", "\n-----------------"
#                 ]))
#               except KeyError:
#                 next

#             return '\n'.join(string)

# NewsSearch = SearchNews()

In [41]:
# NewsSearch.run("NQ")

### SECTools:

In [42]:
# def download_form_html(url):
#   headers = {
#     'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#     'Accept-Encoding': 'gzip, deflate, br',
#     'Accept-Language': 'en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7',
#     'Cache-Control': 'max-age=0',
#     'Dnt': '1',
#     'Sec-Ch-Ua': '"Not_A Brand";v="8", "Chromium";v="120"',
#     'Sec-Ch-Ua-Mobile': '?0',
#     'Sec-Ch-Ua-Platform': '"macOS"',
#     'Sec-Fetch-Dest': 'document',
#     'Sec-Fetch-Mode': 'navigate',
#     'Sec-Fetch-Site': 'none',
#     'Sec-Fetch-User': '?1',
#     'Upgrade-Insecure-Requests': '1',
#     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
#   }

#   response = requests.get(url, headers=headers)

#   return response.text

In [43]:
# def embedding_search(url, ask):
#   text = download_form_html(url)
#   elements = partition_html(text=text)
#   content = "\n".join([str(el) for el in elements])
#   text_splitter = CharacterTextSplitter(
#       separator = "\n",
#       chunk_size = 1000,
#       chunk_overlap  = 150,
#       length_function = len,
#       is_separator_regex = False,
#   )
#   docs = text_splitter.create_documents([content])
#   retriever = FAISS.from_documents(
#       docs, AzureOpenAIEmbeddings()
#     ).as_retriever()
#   retriever = FAISS.from_documents(
#     docs, client.embeddings
#   ).as_retriever()
#   answers = retriever.get_relevant_documents(ask, top_k=4)
#   answers = "\n\n".join([a.page_content for a in answers])

#   return answers

In [44]:
# class PropertiesSearch10q(BaseModel):
#     stock: str = Field(..., description="A random stock.")
#     ask: str = Field(..., description="Question from the 10-Q of the stock")


# class Search10q(BaseTool):
#     name = "Search10q"
#     description = """Useful to search information from the latest 10-Q form for a given stock.
#                      The input to this tool should be a pipe (|) separated text of
#                      length two, representing the stock ticker you are interested and what
#                      question you have from it.
#                      For example, `AAPL|what was last quarter's revenue`."""

#     args_schema: Type[BaseModel] = PropertiesSearch10q

#     def _run(
#         self, stock: str, ask: str
#     ) -> dict:
#             queryApi = QueryApi(api_key="befd094ddadf6a9e080bf194b6d9197e753ec1d4226fc8e595ed717b2f1bc3a5")
#             query = {
#               "query": {
#                 "query_string": {
#                   "query": f"ticker:{stock} AND formType:\"10-Q\""
#                 }
#               },
#               "from": "0",
#               "size": "1",
#               "sort": [{ "filedAt": { "order": "desc" }}]
#             }

#             fillings = queryApi.get_filings(query)['filings']
#             if len(fillings) == 0:
#               return "Sorry, I couldn't find any filling for this stock, check if the ticker is correct."
#             link = fillings[0]['linkToFilingDetails']
#             answer = embedding_search(link, ask)
#             return answer

# TenQ = Search10q()

In [45]:
# TenQ.run({"stock":"AAPL", "ask":"what was last quarter's revenue."})

In [46]:
# class PropertiesSearch10k(BaseModel):
#     stock: str = Field(..., description="A random stock.")
#     ask: str = Field(..., description="Question from the 10-k of the stock")


# class Search10k(BaseTool):
#     name = "Search10k"
#     description = """Useful to search information from the latest 10-K form for a given stock.
#                      The input to this tool should be a pipe (|) separated text of
#                      length two, representing the stock ticker you are interested, what
#                      question you have from it.
#                      For example, `AAPL|what was last year's revenue`."""

#     args_schema: Type[BaseModel] = PropertiesSearch10k

#     def _run(
#         self, stock: str, ask: str
#     ) -> dict:
#             queryApi = QueryApi(api_key="befd094ddadf6a9e080bf194b6d9197e753ec1d4226fc8e595ed717b2f1bc3a5")
#             query = {
#               "query": {
#                 "query_string": {
#                   "query": f"ticker:{stock} AND formType:\"10-K\""
#                 }
#               },
#               "from": "0",
#               "size": "1",
#               "sort": [{ "filedAt": { "order": "desc" }}]
#             }

#             fillings = queryApi.get_filings(query)['filings']
#             if len(fillings) == 0:
#               return "Sorry, I couldn't find any filling for this stock, check if the ticker is correct."
#             link = fillings[0]['linkToFilingDetails']
#             answer = embedding_search(link, ask)
#             return answer

# Tenk = Search10k()

## Openai utilites

In [47]:
import os
from openai import AzureOpenAI

In [48]:
api_type = "azure"
api_endpoint = 'https://tensurfbrain1.openai.azure.com/'
api_version = '2023-10-01-preview'
api_key = '80ddd1ad72504f2fa226755d49491a61'
client = AzureOpenAI(
    api_key= api_key,
    api_version= api_version,
    azure_endpoint= api_endpoint
)

In [49]:
class ChatWithOpenai:
  def __init__(self, system_message, model, temperature, max_tokens, client, default_user_messages=None):
    self.system_message = system_message
    self.model = model
    self.temperature = temperature
    self.max_tokens = max_tokens
    self.client = client
    self.messages = [{"role": "system", "content": system_message}]
    if default_user_messages:
      for user_message in default_user_messages:
        self.messages += [{"role": "user", "content": user_message}]

  def chat(self, user_input):
    # print(self.messages + user_input)
    response = self.client.chat.completions.create(
        model=self.model,
        messages=self.messages + user_input,
        temperature=self.temperature,
        max_tokens=self.max_tokens
    )
    return response.choices[0].message.content

## Irrelevant handler

In [50]:
# message = "'check the grammer for me?'"
# user_input = [{"role": "user", "content": message}]
# handler_zero_openai.chat(user_input)

In [51]:
# Handler.run("check the grammer for me?")

In [52]:
default_message = ["Check the following text for greeting words and do as the system message said."]

In [53]:
handler_zero_openai = ChatWithOpenai(system_message="You are an assistant. Your job is to check the user input Not answer it.\
If the user input contains greeting words like ‘hello’, ‘hi’, and so on, \
then you should remove these words. \
Note that the final response is either the input with the greeting word removed, \
or ‘None’ if the input consists only of a greeting word. \
if there is no greeting word in the input, the response is the last user input itself \
without any changes. \
No other responses are possible. \
Here are some examples and the valid responses: \
Example 1 (when there is no greeting word): \
{ input: ‘What is the weather?’ response: ‘What is the weather?’ } \
Example 2 (when there is a greeting word): \
{ input: ‘Hi can you speak French?’ response: ‘Can you speak French?’ } \
Example 3 (when there is just one or more greeting words): \
{ input: ‘Hello.’ response: ‘None’ } \
Note that you are not permitted to answer user requests directly. \
Only perform the tasks instructed by system messages.",

                                      default_user_messages=default_message,
                                      model="gpt_35_16k",
                                      temperature=0.2,
                                      max_tokens=100,
                                      client=client)

In [54]:
handler_one_openai = ChatWithOpenai(system_message="You are an assistant. Your job is to check the user input, not answer it. \
We are a system with the name 'Tensurf Brain' or 'Tensurf'. \
If the user needs any information about us or needs a tutorial for \
how our system is working, your job is to detect these scenarios and respond with 'True'. \
If the user asks about you, the answer is also 'True'. \
Also, when the user is confused and doesn't know how to start or what to do, \
the answer is also 'True'. \
For any other input that doesn't classify in the tutorial or information \
about 'Tensurf Brain' or 'Tensurf' you should return false. \
Note that you are not permitted to answer user requests directly. \
Only perform the tasks instructed by system messages.",
                                    model="gpt_35_16k",
                                    temperature=0.2,
                                    max_tokens=100,
                                    client=client)

In [55]:
handler_two_openai = ChatWithOpenai(system_message="You are an assistant. Your job is to check the user input. \
If the user input does not contain any financial and \
trading topics or requests, then you should answer only\
with ‘True’. Otherwise, return ‘False’. \
Possible responses for you are 'True' or 'False'. \
For example, the correct response to the question \
'What is the trend?' is 'False'.",
                              model="gpt_35_16k",
                              temperature=0,
                              max_tokens=100,
                              client=client)

In [56]:
class HandleIrrelevantSchema(BaseModel):
    massage: str = Field(..., description="The humanmassage")


class HandleIrrelevant(BaseTool):
    name = "HandleIrrelevant"
    description = """This function checks if the message contains financial or trading subjects or not. \
The output of this function is either True or False and the possible output of this function are 'True' or 'False'.: \
True: when the message contains financial or trading subjects. \
False: when the message request is not in these fields."""

    args_schema: Type[BaseModel] = HandleIrrelevantSchema

    def _run(
        self, massage: str
    ) -> dict:

        user_input = [{"role": "user", "content": "'" + massage + "'"}]

        modified_input = handler_zero_openai.chat(user_input)
        # print(f"inside handler func: {modified_input}")
        if modified_input == 'None':
          return 'Greeting'

        modified_user_input = [{"role": "user", "content": modified_input}]

        if handler_one_openai.chat(modified_user_input) == 'True':
          return "Tutorial"
        # print(f"in the handler: {modified_user_input}")
        else:
          return handler_two_openai.chat(modified_user_input)

        # chat_with_openai(user_input)
        # return chat_with_openai(user_input)

Handler = HandleIrrelevant()

In [57]:
# Handler.run("What is the trend?")

In [58]:
# tools = [trend, SR, TP, Sl, Bias, databaseId, databaseRisk, databaseStyle, Handler]

In [59]:
tools = [Trend, SR, TP, SL, Bias, Handler]

In [60]:
tool_executor = ToolExecutor(tools)

## Helper functions:

In [61]:
from langchain_core.messages import AIMessage, BaseMessage, ChatMessage, FunctionMessage, HumanMessage
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.agents import AgentAction, AgentFinish
from langgraph.graph import END, StateGraph
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation
from langchain_core.tools import tool
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union
from langchain.agents import create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, BasePromptTemplate, MessagesPlaceholder
import operator
import functools
import json

In [62]:
def create_agent(llm, tools, system_message: str):
  """Create an agent."""
  functions = [format_tool_to_openai_function(t) for t in tools]
  prompt = ChatPromptTemplate.from_messages(
      [
          (
              "system",
              " You are a helpful AI assistant, collaborating with other assistants."
              " Use the provided tools to progress towards answering the question."
              " If you are unable to fully answer, that's OK, another assistant with different tools "
              " will help where you left off. Execute what you can to make progress."
              " If you or any of the other assistants have the final answer or deliverable,"
              " prefix your response with FINAL ANSWER so the team knows to stop."
              f" You have access to the following tools: {tool}.\n{system_message}",
          ),
          MessagesPlaceholder(variable_name="messages"),
      ]
  )
  prompt = prompt.partial(system_message=system_message)
  prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
  return prompt | llm.bind_functions(functions)

In [63]:
def agent_node(state, agent, name):
  result = agent.invoke(state)

  if isinstance(result, FunctionMessage):
      pass

  else:
      result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)

  return {
      "messages": [result],
      "sender": name,
      "output_json": state["output_json"]
  }

## Agents:

In [64]:
from langchain_core.agents import AgentActionMessageLog
from langchain_openai import AzureChatOpenAI

In [65]:
api_type = "azure"
api_endpoint = 'https://tensurfbrain1.openai.azure.com/'
api_version = '2023-10-01-preview'
api_key = '80ddd1ad72504f2fa226755d49491a61'

In [66]:
llm = AzureChatOpenAI(
                  api_key=api_key,
                  api_version=api_version,
                  azure_endpoint=api_endpoint,
                  deployment_name="gpt_35_16k",
                  temperature=0,
                  streaming=True
)

In [67]:
# Trend:At any situations, never return the number which is the output of the Trend function. Instead, use its correcsponding explanation which is in the Trend tool's description. Make sure to mention the start_datetime and end_datetime or the lookback parameter if the user have mentioned in their last message. If the user provide neither specified both start_datetime and end_datetime nor lookback parameters, politely tell them that they should and introduce these parameters to them so that they can use them. Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Now generate a proper response. \
trading_agent = create_agent(
    llm,
    [Trend, SR, TP, SL, Bias],
    system_message="""You are an assistant with the following capabilities given to you by your tools:
SR: Calculate support and resistance levels. \
Trend: Calculate the trend of a specified financial instrument over a given time range and timeframe. \
TP: Calculate Take profit (TP), \
SL: Calculate Stoploss (SL), \
Bias: Detecting trading bias. \
Note the following rules for result of each tool: \
SR:Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Do not mention the name of the levels that the level is support or resistance. The final answer should also contain the following texts: These levels are determined based on historical price data and indicate areas where the price is likely to encounter support or resistance. The associated scores indicate the strength or significance of each level, with higher scores indicating stronger levels. \
Trend:At any situations, never return the number which is the output of the Trend function. Instead, use its correcsponding explanation which is in the Trend tool's description. Make sure to mention the start_datetime and end_datetime or the lookback parameter. Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Now generate a proper response. \
TP: Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Now generate a proper response. \
SL: Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. The unit of every number in the answer should be mentioned. Now generate a proper response. \
Bias: Do not mention the name of the parameters of the functions directly in the final answer. Instead, briefly explain them and use other meaningfuly related synonyms. Now generate a proper response."""
)
trading_node = functools.partial(agent_node, agent=trading_agent, name="Trading")

c:\Users\Sepehr\anaconda3\envs\sepehr\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


In [68]:
# onboarding_agent = create_agent(
#     llm,
#     [databaseId, databaseRisk, databaseStyle],
#     """You are an onboarding assistant who helps new users set and change some of the user fields in the database.
#        Note that user requests in our systems must only pertain to financial and trading topics.
#        If the user’s request is not related to these topics, this tool will handle it.""",
# )
# onboarding_node = functools.partial(agent_node, agent=onboarding_agent, name="Onboarding")

## Graph

### adding sub graph

In [69]:
import uuid

In [70]:
def reduce_list(left: list | None, right: list | None) -> list:
  """Append the right-hand list, replacing any elements with the same id in the left-hand list."""
  if not left:
      left = []
  if not right:
      right = []
  left_, right_ = [], []
  for orig, new in [(left, left_), (right, right_)]:
      for val in orig:
          if not isinstance(val, dict):
              val = {"val": val}
          if "id" not in val:
              val["id"] = str(uuid.uuid4())
          new.append(val)
  # Merge the two lists
  left_idx_by_id = {val["id"]: i for i, val in enumerate(left_)}
  merged = left_.copy()
  for val in right_:
      if (existing_idx := left_idx_by_id.get(val["id"])) is not None:
          merged[existing_idx] = val
      else:
          merged.append(val)
  return merged

In [71]:
# class AgentState(TypedDict):
#   messages: Annotated[Sequence[BaseMessage], operator.add]
#   path: Annotated[list[str], reduce_list]
#   # sender: str

In [72]:
# subGraphOne = StateGraph(AgentState)
# mainGraph = StateGraph(AgentState)

# mainGraph.add_node("Handler", run_Handler)

# subGraphOne.add_node("Greeting", run_greeting)
# subGraphOne.add_node("Tutorial", run_tutorial)

# subGraphOne.add_edge("Greeting", END)
# subGraphOne.add_edge("Tutorial", END)
# subGraphOne.add_conditional_edges(
#     "Handler",
#     router,
#     {"Greeting": "Greeting", "Tutorial": "Tutorial", "end": END},
# )

# subGraphOne.set_entry_point("Handler")
# # subGraphOne.set_finish_point("Trading")

# mainGraph.add_node("Trading", trading_node)
# mainGraph.add_node("Onboarding", onboarding_node)
# mainGraph.add_node("call_tool", tool_node)
# mainGraph.add_node("child", subGraphOne.compile())

# mainGraph.add_conditional_edges(
#     "Trading",
#     router,
#     {"continue": "Trading", "call_tool": "call_tool", "end": END},
# )
# mainGraph.add_conditional_edges(
#     "Onboarding",
#     router,
#     {"continue": "Trading", "call_tool": "call_tool", "end": END},
# )


# mainGraph.add_conditional_edges(
#     "call_tool",
#     lambda x: x["sender"],
#     {
#         "Trading": "Trading",
#         "Onboarding": "Onboarding",
#     },
# )
# mainGraph.set_entry_point("Handler")

# graph = mainGraph.compile()

### normal

In [73]:
def run_Handler(state):
  tool_name = "HandleIrrelevant"

  messages = state["messages"]
  last_message = messages[-1]

  # print("&"*50)
  # print(last_message.content)
  # print("&"*50)

  action = ToolInvocation(
      tool=tool_name,
      tool_input=last_message.content,
  )

  response = tool_executor.invoke(action)
  # print(response)
  standard_response = ""
  if response == 'True':
    standard_response = "I'm here to help with trading and financial market queries. If you think your ask relates to trading and isn't addressed, please report a bug using the bottom right panel."

  function_message = FunctionMessage(
       content=standard_response, HandleIrrelevant_response=f"{str(response)}", name=action.tool
  )

  return {"messages": [function_message]}

In [74]:
def run_greeting(state):
  # tool_name = "HandleIrrelevant"

  # action = ToolInvocation(
  #     tool=tool_name,
  #     tool_input="Hi",
  # )

  # response = tool_executor.invoke(action)

  greeting = ChatWithOpenai(system_message="You are an financial and trading assistant.",
                            model="gpt_35_16k",
                            temperature=0.2,
                            max_tokens=100,
                            client=client)
  input = [{"role": "user", "content": "Hi"}]
  response = greeting.chat(input)

  function_message = FunctionMessage(
       content=response ,name='openai chat'
  )

  return {"messages": [function_message]}

In [75]:
def run_tutorial(state):
  response = """I'm TenSurf Brain, your AI trading assistant within TenSurf Hub platform, designed to enhance your trading experience with advanced analytical and data-driven tools:
1. Trend Detection: I can analyze and report the trend of financial instruments over your specified period. For example, ask me, "What is the trend of NQ stock from May-1-2024 12:00:00 until May-5-2024 12:00:00?"
2. Support and Resistance Levels: I identify and score key price levels that may influence market behavior based on historical data. Try, "Calculate Support and Resistance Levels based on YM by looking back up to the past 10 days and a timeframe of 1 hour."
3. Stop Loss Calculation: I determine optimal stop loss points to help you manage risk effectively. Query me like, "How much would be the optimal stop loss for a short trade on NQ?"
4. Take Profit Calculation: I calculate the ideal exit points for securing profits before a potential trend reversal. For example, "How much would be the take-profit of a short position on Dow Jones with the stop loss of 10 points?"
5. Trading Bias Identification: I analyze market conditions to detect the best trading biases and directions, whether for long or short positions. Ask me, "What is the current trading bias for ES?"
Each tool is tailored to help you make smarter, faster, and more informed trading decisions. Enjoy!"""

  function_message = FunctionMessage(
       content=response ,name='hardcoded string'
  )

  return {"messages": [function_message]}

In [76]:
def output_json_assigner(tool_name, response):
  output_json = {}
  if tool_name == "calculate_sr":
    sr_value, sr_start_date, sr_detect_date, sr_end_date, sr_importance = response
    output_json["levels_prices"] = sr_value
    output_json["levels_start_timestamps"] = sr_start_date
    output_json["levels_detect_timestamps"] = sr_detect_date
    output_json["levels_end_timestamps"] = sr_end_date
    output_json["levels_scores"] = sr_importance
  return output_json

In [77]:
def tool_node(state):
  """This runs tools in the graph
      It takes in an agent action
      and calls that tool and
      returns the result."""
  messages = state["messages"]
  last_message = messages[-1]
  tool_input = json.loads(
      last_message.additional_kwargs["function_call"]["arguments"]
  )

  if len(tool_input) == 1 and "__arg1" in tool_input:
      tool_input = next(iter(tool_input.values()))
  # tool_name = last_message.additional_kwargs["function_call"]["name"]

  tool_name = last_message.additional_kwargs["function_call"]["name"].split(".")[-1]

  action = ToolInvocation(
      tool=tool_name,
      tool_input=tool_input,
  )

  response = tool_executor.invoke(action)

  out_json = output_json_assigner(tool_name, response)


  function_message = FunctionMessage(
      content=f"{tool_name} response: {str(response)}", name=action.tool
  )

  return {"messages": [function_message], "output_json":out_json}

In [78]:
def router(state):
    messages = state["messages"]
    last_message = messages[-1]

    if last_message.name != 'HandleIrrelevant':
      last_message = messages[-2]

      if "function_call" in last_message.additional_kwargs:
        return "call_tool"

      if not "function_call" in last_message.additional_kwargs and last_message.type == "function":
        return "continue"

      if not last_message.additional_kwargs and last_message.type == "human":
        return "end"

    else:
      if "Greeting" in last_message.HandleIrrelevant_response and last_message.name == 'HandleIrrelevant':
        return "Greeting"

      if "Tutorial" in last_message.HandleIrrelevant_response and last_message.name == 'HandleIrrelevant':
        return "Tutorial"

      if "True" in last_message.HandleIrrelevant_response and last_message.name == 'HandleIrrelevant':
        return "end"

      if "False" in last_message.HandleIrrelevant_response and last_message.name == 'HandleIrrelevant':
        return "continue"

      if "False" in last_message.content and last_message.name == "HandleGreeting":
        return "continue"

      if "False" not in last_message.content and last_message.name == "HandleGreeting":
        return "end"


    return "continue"

In [79]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]
  output_json: dict
  input_json: dict
  sender: str

In [80]:
workflow = StateGraph(AgentState)

workflow.add_node("Trading", trading_node)
# workflow.add_node("Onboarding", onboarding_node)
workflow.add_node("call_tool", tool_node)
workflow.add_node("Handler", run_Handler)
workflow.add_node("Greeting", run_greeting)
workflow.add_node("Tutorial", run_tutorial)

# workflow.add_node("Inquiry", inquiry_node)
# workflow.add_node("Support", support_node)
# workflow.add_node("Inquiry", inquiry_node)

# workflow.add_edge("Inquiry", "call_tool")

# workflow.add_edge("Handler", "Trading")

workflow.add_edge("Greeting", END)
workflow.add_edge("Tutorial", END)

# workflow.add_conditional_edges(
#     "Inquiry",
#     router,
#     {"call_tool": "call_tool", "continue": "Inquiry"},
# )
# workflow.add_conditional_edges(
#     "Support",
#     router,
#     {"continue": "Trading", "call_tool": "call_tool", "end": END},
# )
workflow.add_conditional_edges(
    "Handler",
    router,
    {"continue": "Trading", "Greeting": "Greeting", "Tutorial": "Tutorial", "end": END},
)
# workflow.add_conditional_edges(
#     "Greeting",
#     router,
#     {"continue": "Trading", "end": END},
# )
workflow.add_conditional_edges(
    "Trading",
    router,
    {"continue": "Trading", "call_tool": "call_tool", "end": END},
)
# workflow.add_conditional_edges(
#     "Onboarding",
#     router,
#     {"continue": "Trading", "call_tool": "call_tool", "end": END},
# )


workflow.add_conditional_edges(
    "call_tool",
    lambda x: x["sender"],
    {
        "Trading": "Trading",
        # "Onboarding": "Onboarding",
        # "Inquiry": "Inquiry",
        # "Support": END
    },
)
workflow.set_entry_point("Handler")

graph = workflow.compile()

## Graph visualization

In [81]:
from IPython.display import display, HTML
import base64

In [82]:
def display_image(image_bytes: bytes, width=300):
    decoded_img_bytes = base64.b64encode(image_bytes).decode("utf-8")
    html = f'<img src="data:image/png;base64,{decoded_img_bytes}" style="width: {width}px;" />'
    display(HTML(html))

In [83]:
# display_image(graph.get_graph().draw_png())

## Test prompts:

In [84]:
# sample prompt for each function 
prompts = [ 
    # # detect_trend 
    # "What is the trend of NQ stock from May-1-2024 12:00:00 until May-5-2024 12:00:00?", 
    # calculate_sr 
    "Calculate Support and Resistance Levels based on YM by looking back up to past 5 days and timeframe of 10 minutes.", 
    # calculate_sl 
    "What would be the stop loss of trading short positinos based on NQ and minmax method by looking back up to 30 candles?", 
    # calculate_tp 
    "How much would be the take-profit of the NQ with the stop loss of 10 and direction of 1?", 
    # bias_detection 
    "Tell me about the bias of ES on the market.",
	# irrelevant
	"What are the rules of basketball?"
]

for prompt in prompts:
	print(f"Prompt:  	{prompt}") 
	for s in graph.stream( 
			{ 
				"messages": [ 
					HumanMessage( 
					content=prompt 
					) 
				], 
				"input_json": input_filter.front_end_json_sample 
			}, 
			# Maximum number of steps to take in the graph 
			{"recursion_limit": 150}, 
		): 
		print(s)

Prompt:  	Calculate Support and Resistance Levels based on YM by looking back up to past 5 days and timeframe of 10 minutes.
{'Handler': {'messages': [FunctionMessage(content='', name='HandleIrrelevant', HandleIrrelevant_response='False')]}}
{'Trading': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "symbol": "YM",\n  "lookback_days": "5",\n  "timeframe": "10min"\n}', 'name': 'calculate_sr'}}, response_metadata={'finish_reason': 'function_call'}, name='Trading', id='run-009e8632-f9c4-4fec-b0cf-d690cf3c35bb-0', tool_calls=[], invalid_tool_calls=[])], 'output_json': None, 'sender': 'Trading'}}
{'Trading': {'messages': [HumanMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "symbol": "YM",\n  "lookback_days": "5",\n  "timeframe": "10min"\n}', 'name': 'calculate_sr'}}, response_metadata={'finish_reason': 'function_call'}, name='Trading', id='run-ff1eab67-fd69-4c20-940e-1888def84e29-0', tool_calls=[], invalid_tool_cal